Try arima
serima
autoarima 
auto sarima
train + test model upload 

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose

from tqdm import tqdm
# from utils.metrics import evaluate
# warning supression
import warnings
warnings.filterwarnings("ignore")


# Models
import pmdarima as pm
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA, ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX


# Extra setting for charts
from pylab import rcParams
plt.style.use('bmh')
# seed = 44
# np.random.seed(seed)
# mpl.rcParams['axes.labelsize'] = 14
# mpl.rcParams['xtick.labelsize'] = 12
# mpl.rcParams['ytick.labelsize'] = 12
# mpl.rcParams['text.color'] = 'k'

In [17]:
# Importing data for univariate time series analysis
df =  pd.read_csv('train.csv', parse_dates =[['date', 'hour']])
df.set_index('date_hour', inplace = True)
df

,demand
date_hour,
2018-08-18 9,91
2018-08-18 10,21
2018-08-18 13,23
2018-08-18 14,104
2018-08-18 15,81
...,...
2021-02-28 19,95
2021-02-28 20,88
2021-02-28 21,39


In [ ]:
plt.figure(num=None, figsize=(30, 10), dpi=80, facecolor='w', edgecolor='k')
plt.title('CAR Data', fontsize=30)

plt.plot(df['demand'])

In [18]:
#  Test for stationary series

from statsmodels.tsa.stattools import adfuller
X = df['demand'].values
result = adfuller(X)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

if result[0] < result[4]["5%"]:
    print ("Reject Ho - Time Series is Stationary")
else:
    print ("Failed to Reject Ho - Time Series is Non-Stationary")

ADF Statistic: -12.543617
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Reject Ho - Time Series is Stationary


>So our Series is stationary

In [5]:
decompose = seasonal_decompose(df, model = 'multiplicative', freq = 12)
rcParams['figure.figsize'] = 10,5
decompose.plot()
# pyplot.f

NameError: name 'seasonal_decompose' is not defined

In [6]:
# Train test split
train = df[0:16000]
test = df[16000:]

,demand
date_hour,
2020-10-19 20,17
2020-10-19 21,61
2020-10-19 22,39
2020-10-19 23,51
2020-10-20 0,36
...,...
2021-02-28 19,95
2021-02-28 20,88
2021-02-28 21,39


## Model training

In [22]:
resultsDict = []
predictionsDict = []

In [21]:
autoModel = pm.auto_arima(train['demand'], trace=True,
                          error_action='ignore', suppress_warnings=True, seasonal=False) 
autoModel.fit(train['demand'])

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=159458.885, Time=32.48 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=165855.680, Time=0.60 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=162322.388, Time=1.24 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=160787.398, Time=3.60 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=165853.680, Time=0.32 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=160787.115, Time=7.21 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=159591.824, Time=12.59 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=159418.629, Time=30.62 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=159417.555, Time=16.66 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=160996.604, Time=1.93 sec
 ARIMA(4,1,1)(0,0,0)[0] intercept   : AIC=159418.010, Time=26.66 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=161252.223, Time=1.28 sec
 ARIMA(4,1,0)(0,0,0)[0] intercept   : AIC=160904.304, Time=2.44 sec
 ARIMA(4,1,2)(0,0,0)[0] intercept   : AIC=159421.376, Time=23.32 sec

ARIMA(order=(3, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [ ]:
order = autoModel.order
yhat = list()
for t in tqdm(range(len(test['demand']))):
    temp_train = df[:len(train)+t]
    model = ARIMA(temp_train['demand'], order=(3,0,1))
    model_fit = model.fit(disp=False)
    predictions = model_fit.predict(
        start=len(temp_train), end=len(temp_train), dynamic=False)
    yhat = yhat + [predictions]

yhat = pd.concat(yhat)
# resultsDict['AutoARIMA {0}'.format(order)] = evaluate(
#     df_test.pollution_today, yhat)
predictionsDict['AutoARIMA {0}'.format(order)] = yhat.values

  1%|█                                                                             | 30/2247 [07:41<9:04:16, 14.73s/it]

In [27]:
# 3,1,1 order

In [3]:
plt.plot(df_test.pollution_today.values, label='Original')
plt.plot(yhat.values, color='red', label='AutoARIMA {0}'.format(order))
plt.legend()

NameError: name 'plt' is not defined

In [14]:
!pip install pmdarima